<a href="https://colab.research.google.com/github/ummadiviany/Pix2Pix/blob/main/Training_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pix2Pix Training Notebook



## Installing necessary libraries

In [1]:
!pip install -q albumentations==0.4.6   # Albumentations for data augumentation 
!pip install -q opendatasets    # To download datasets

     |████████████████████████████████| 117 kB 10.1 MB/s 
     |████████████████████████████████| 948 kB 46.1 MB/s 


## Importing necessary libraries
- You should be able to see **Successfully imported all libraries**

In [2]:
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader, Dataset
    import tqdm as tqdm
    from torchvision.utils import save_image, make_grid
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    import os
    import numpy as np
    from PIL import Image # Image reading 
    from torchvision import datasets
    import matplotlib.pyplot as plt # Image display
    import opendatasets as od # dataset download
    %matplotlib inline  
    import pandas as pd # for creating Loss dataframe
    
    print("Successfully imported all libraries")
except:
    print("Errors in importing libraries")


Successfully imported all libraries


## Cloning git repo for model functions
- I have all the model and additional function classes stored in my github

In [5]:
!git clone https://github.com/ummadiviany/Pix2Pix

fatal: destination path 'Pix2Pix' already exists and is not an empty directory.


In [6]:
from Pix2Pix.training_notebooks.generator_model import Generator
from Pix2Pix.training_notebooks.discriminator_model import Discriminator
from Pix2Pix.training_notebooks.dataset import MapDataset
from Pix2Pix.training_notebooks.additional_functions import test_on_val_data

## Datasets download - Attention Needed
Use the below kaggle usename and key for dataset download
1. Below code cell prompts for kaggle username, copy the username from below and paste and hit ⌨Enter key.
2. Again prompts for kaggle secure key, copy the key from below and paste and hit ⌨Enter key.
3. It will take about ~2min⏲ to download the datasets
- username ▶     **iamvinayummadi** 
- key:     ▶    **78f6cee94760fd02415c9024cba10173**

In [ ]:
od.download('https://www.kaggle.com/vikramtiwari/pix2pix-dataset')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: iamvinayummadi
Your Kaggle Key: ··········


 67%|██████▋   | 1.61G/2.40G [00:41<00:16, 52.0MB/s]

## Setting up hyperparameters

1.   Change the **NUM_EPOCHS=2** if needed
2.   Change **BATCH_SIZE = 32** if needed



In [ ]:
NUM_EPOCHS = 2
loss_df = pd.DataFrame(columns=['D_Loss','G_Loss'])
LEARNING_RATE  = 3e-4
BATCH_SIZE = 32
NUM_WORKERS = 2
IMAGE_SIZE = 256
CHANNELS_IMG = 3
L1_LAMBDA = 100
LAMBDA_GP = 10
TRAIN_DIR = "pix2pix-dataset/maps/maps/train"
VAL_DIR = "pix2pix-dataset/maps/maps/val"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device :',DEVICE)

## Loading training and validation data

In [ ]:
train_dataset = MapDataset(root_dir= TRAIN_DIR, input_size=600,direction=0)
train_loader = DataLoader(train_dataset,batch_size= BATCH_SIZE,shuffle=True,num_workers= NUM_WORKERS)

val_dataset  = MapDataset(root_dir= VAL_DIR, input_size=600,direction=0)
validation_loader = DataLoader(val_dataset,batch_size=4,shuffle=True)

## Model instances, optimizers, learning_rate schedulers, and loss functions
1. Adam optimizer(lr = 2e-4,betas=(0.5,0.99) with stepwise learning rate decay is used. Learning rate decay by factor of 10 for every 20 epochs.
2. BCE Loss for Discriminator and BCE + L1 Loss for Generator

In [ ]:
disc = Discriminator(in_channels=3).to( DEVICE)
gen_model = Generator(in_channels=3,features=64).to( DEVICE)

opt_disc = optim.Adam(disc.parameters(),lr= LEARNING_RATE,betas=(0.5,0.999))
opt_gen = optim.Adam(gen_model.parameters(),lr= LEARNING_RATE,betas=(0.5,0.999))

scheduler_disc = optim.lr_scheduler.StepLR(opt_disc, step_size=20, gamma=0.1)
scheduler_gen = optim.lr_scheduler.StepLR(opt_gen, step_size=20, gamma=0.1)

BCE = nn.BCEWithLogitsLoss()
L1_LOSS = nn.L1Loss()

## Training loop
* Prints Epoch, Batch, Discriminator Loss, Generator Loss
* Saves an Image📺 with name format input_label_gen_.png for visualization. Please check that image📺

In [ ]:
for epoch in range( NUM_EPOCHS):
    print(f"Epoch[{epoch}/{NUM_EPOCHS}], Learning Rate = {opt_disc.param_groups[0]['lr']}") # printing learning rate
    for idx,(inputs,outputs) in enumerate(train_loader):    #enumerating thorugh train-dataset
        inputs,outputs=inputs.to( DEVICE), outputs.to( DEVICE)  # sending to GPU
        
        #Train Discriminator
        outputs_fake = gen_model(inputs)    #Generating translated images
        D_real = disc(inputs,outputs)       # Discriminator call on inputs and outputs ones
        D_real_loss = BCE(D_real,torch.ones_like(D_real))   # Calculates loss value
        D_fake = disc(inputs,outputs_fake.detach())         # Discriminator call on inputs and genrated ones
        D_fake_loss = BCE(D_fake,torch.zeros_like(D_fake))  # Calculates loss value
        D_loss = (D_real_loss+D_fake_loss)/2                # Aggeregate loss
        opt_disc.zero_grad()                # clearing optimizer gradients
        D_loss.backward()                   # Backward function call
        opt_disc.step()                     # Taking one optimizer step
        
        # Train Generator
        D_fake = disc(inputs,outputs_fake)  # Discriminator call on inputs and genrated ones
        G_fake_loss = BCE(D_fake,torch.ones_like(D_fake))   # Calculates loss value
        L1 = L1_LOSS(outputs_fake,outputs)* L1_LAMBDA       # Calculates loss value
        G_loss = G_fake_loss+L1             # Generator loss
        opt_gen.zero_grad()                 # clearing optimizer gradients
        G_loss.backward()                   # Backward function call
        opt_gen.step()                      # Taking one optimizer step
        
        loss_df.loc[len(loss_df)] = [D_loss.mean().item(),G_loss.mean().item()]        # save loss value in dataframe row
        loss_df.to_csv('losses.csv',index=False)                                       # write datafram file to disk
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}]  Batch [{idx+1}/{len(train_loader)}] PatchGAN_Loss : {D_loss.mean().item():.4f}  Generator_Loss : {G_loss.mean().item():.4f}")
        test_on_val_data(epoch, "/content/", gen_model, validation_loader, DEVICE)
        
    print('See the generated image at/content/input_label_gen_.png')
    # Learning rate update with LR Scheduler
    scheduler_disc.step()       # take one scheduler step
    scheduler_gen.step()        # take one scheduler step

## Visualising results
- Let's👀see how are results at second epoch. Network needs to train for more than 250 epoch to get better results.

In [ ]:
def visualize(image):
    img = Image.open(image)
    plt.figure(figsize=(15,20))
    plt.title('1st Row = Input Image,   2nd Row = Target Image,     3rd Row = Translated Image')
    plt.axis('off')
    plt.imshow(img)
    plt.show()

visualize('input_label_gen_2.png')

## Plotting Loss values

In [ ]:
df_loss = pd.read_csv('losses.csv')
plt.plot(df_loss['D_Loss'],label='PatchGAN Loss')
plt.plot(df_loss['G_Loss'],label='Generator Loss')
plt.xlabel('No of Batch Iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()